In [38]:
import pandas as pd

data = pd.read_csv('datasets/twitter/train.csv', encoding='ISO-8859-1')
print(data.head())

   ItemID  Sentiment                                      SentimentText
0       1          0                       is so sad for my APL frie...
1       2          0                     I missed the New Moon trail...
2       3          1                            omg its already 7:30 :O
3       4          0            .. Omgaga. Im sooo  im gunna CRy. I'...
4       5          0           i think mi bf is cheating on me!!!   ...


In [39]:
x_train, y_train = data.SentimentText, data.Sentiment

In [41]:
from keras.layers import Dense, Dropout, Embedding
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.recurrent import GRU
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

t = Tokenizer(num_words=10000)
t.fit_on_texts(x_train)
x_mat = t.texts_to_sequences(x_train)

num_words = 10000

In [43]:
max_len = max([len(s) for s in x_mat])
x_mat = pad_sequences(x_mat, maxlen=max_len)

In [44]:
y_train = np_utils.to_categorical(y_train, 2)

In [49]:
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()
model.add(Embedding(num_words, 128, input_length=125))
model.add(GRU(256, dropout=0.2))
model.add(Dense(2, activation='softmax'))

In [54]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 125, 128)          1280000   
_________________________________________________________________
gru_6 (GRU)                  (None, 256)               295680    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 514       
Total params: 1,576,194
Trainable params: 1,576,194
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.fit(x_mat, y_train, epochs=10, batch_size=1024, validation_split=0.25)

Train on 74991 samples, validate on 24998 samples
Epoch 1/10
74991/74991 [==============================] - 593s 8ms/step - loss: 0.5384 - acc: 0.7299 - val_loss: 0.4945 - val_acc: 0.7572
Epoch 2/10
74991/74991 [==============================] - 705s 9ms/step - loss: 0.4631 - acc: 0.7822 - val_loss: 0.4999 - val_acc: 0.7539
Epoch 3/10
74991/74991 [==============================] - 853s 11ms/step - loss: 0.4374 - acc: 0.7966 - val_loss: 0.4989 - val_acc: 0.7571
Epoch 4/10
74991/74991 [==============================] - 851s 11ms/step - loss: 0.4203 - acc: 0.8060 - val_loss: 0.4826 - val_acc: 0.7700
Epoch 5/10
74991/74991 [==============================] - 592s 8ms/step - loss: 0.4058 - acc: 0.8147 - val_loss: 0.4819 - val_acc: 0.7679
Epoch 6/10
74991/74991 [==============================] - 616s 8ms/step - loss: 0.3913 - acc: 0.8223 - val_loss: 0.4908 - val_acc: 0.7655
Epoch 7/10
74991/74991 [==============================] - 626s 8ms/step - loss: 0.3785 - acc: 0.8306 - val_loss: 0.4916 

In [60]:
test_data = pd.read_csv('datasets/twitter/test.csv', encoding='ISO-8859-1')
print(test_data.head())

   ItemID                                      SentimentText
0       1                       is so sad for my APL frie...
1       2                     I missed the New Moon trail...
2       3                            omg its already 7:30 :O
3       4            .. Omgaga. Im sooo  im gunna CRy. I'...
4       5           i think mi bf is cheating on me!!!   ...


In [61]:
x_test = test_data.SentimentText

In [62]:
t2 = Tokenizer(num_words=10000)
t2.fit_on_texts(x_test)
x_mat = t2.texts_to_sequences(x_test)

num_words = 10000

In [63]:
x_mat = pad_sequences(x_mat, maxlen=max_len)

In [64]:
pred = model.predict(x_mat)

In [81]:
len(x_test)

299989

In [68]:
import numpy as np
for i in range(len(pred)):
    pred[i] = np.argmax(pred[i])

In [71]:
ans = np.zeros((len(pred), 1), dtype=np.int32)
for i in range(len(pred)):
    ans[i][0] = int(pred[i][0])

In [74]:
d = pd.DataFrame(ans)

In [75]:
d.column = ['Sentiment']

In [76]:
d.to_csv('ans.csv')